<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 درخواست کارفرمایان</font>
</h1>

<h3 align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Load DataFrame & Library:
</h3>

In [4]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from jdatetime import datetime
from scipy.stats import ttest_ind
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction import FeatureHasher
from sklearn.ensemble import RandomForestRegressor
import re
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

file_dir = "sales_data.csv"
file_dir2 = "dollar_price.csv"
file_dir3 = "country-cities-data.csv"
pd.set_option("display.max_colwidth", None)

In [5]:

df = pd.read_csv(file_dir, low_memory=False)
df["Screen_Size"] = df["Screen_Size"].str.replace('"', "").astype(float)
df["RAM"] = df["RAM"].str.replace("GB", "").astype(int)
df["Weight"] = df["Weight"].replace(["kgs", "kg"], "", regex=True).astype(float)

df["Date_Shamsi"] = df["Order_Date"].copy()
df["Order_Date"] = df["Order_Date"].apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d").togregorian()
)

df_dollar_price = pd.read_csv(file_dir2, index_col=[0])
df_dollar_price["miladi"] = pd.to_datetime(df_dollar_price["miladi"])

df = pd.merge(
    df,
    df_dollar_price,
    how="left",
    left_on="Order_Date",
    right_on="miladi",
)
df.drop(columns=["miladi", "shamsi"], inplace=True)
df.rename(columns={"close_price": "Dollar_Price"}, inplace=True)

<h3 align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Request 6 
</h3>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
شرکت برنامه دارد تا به صورت ماهیانه، بار وارد کشور کند و برای همین می‌خواهد بداند، آیا میزان فروش در ماه‌های مختلف تفاوتی دارد؟
<br>
 در صورت وجود تفاوت به چه نحوی بودجه‌ی سالیانه خود را بین این ماه‌ها تقسیم کند؟
</p>

In [6]:
import pandas as pd
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def test_and_allocate_budget(df):
    df_temp = df.copy()
    df_temp["Month"] = df_temp["Order_Date"].dt.month
    monthly_sales = df_temp.groupby("Month")["Total_Price"].sum()
    statistic, p_value = f_oneway(
        *[group["Total_Price"] for name, group in df_temp.groupby("Month")]
    )
    alpha = 0.05
    yearly_budget = 1
    allocation = [1.0] * 12

    def allocate_budget(significant_pairs, allocation):
        for pair in significant_pairs:
            month1, month2 = int(pair[0]), int(pair[1])
            mean_diff = pair[3]
            allocation[month1 - 1] += mean_diff / 2
            allocation[month2 - 1] -= mean_diff / 2
        return allocation

    if p_value < alpha:
        print("There is a statistically significant difference in sales among different months.")
        tukey_results = pairwise_tukeyhsd(df_temp["Total_Price"], df_temp["Month"])
        if tukey_results.reject.any():
            significant_pairs = tukey_results.summary().data[1:]
            print("Significant pairwise comparisons:")
            allocation = allocate_budget(significant_pairs, allocation)

        allocation = [budget * yearly_budget / sum(allocation) for budget in allocation]
        month_names = [
            "January", "February", "March", "April", "May", "June",
            "July", "August", "September", "October", "November", "December"
        ]
        print("Monthly Budget Allocation:")
        for month, budget in zip(month_names, allocation):
            print(f"{month}: {budget:.2f}")
    else:
        print("There is no statistically significant difference in sales among different months.")

# Example usage
test_and_allocate_budget(df)


There is a statistically significant difference in sales among different months.
Significant pairwise comparisons:
Monthly Budget Allocation:
January: 0.08
February: 0.12
March: 0.08
April: 0.16
May: 0.08
June: 0.00
July: 0.12
August: 0.08
September: 0.08
October: 0.04
November: 0.04
December: 0.08
